<a href="https://colab.research.google.com/github/nccorwin/MSBA-6460-Final-Project/blob/main/6460_Base_BERT_FPB50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers
# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast, TFAutoModel

# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import re
import numpy as np
from sklearn.model_selection import train_test_split

import random

random.seed(42)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 2.7 MB/s 
     |████████████████████████████████| 596 kB 19.8 MB/s 
     |████████████████████████████████| 6.6 MB 17.2 MB/s 
     |████████████████████████████████| 101 kB 3.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
raw = pd.read_csv('/content/drive/MyDrive/Sentences_50Agree.txt', header=None, delimiter="\t", encoding="ISO-8859-1")

In [6]:
raw = raw.rename(columns={0: 'data'})
new = raw["data"].str.split(" .@", n = 1, expand = True)
 
# making separate first name column from new data frame
raw["message"]= new[0]
 
# making separate last name column from new data frame
raw["label"]= new[1]
 
# Dropping old Name columns
raw.drop(columns =["data"], inplace = True)
raw.head()

,message,label
0,"According to Gran , the company has no plans t...",neutral
1,Technopolis plans to develop in stages an area...,neutral
2,The international electronic industry company ...,negative
3,With the new production plant the company woul...,positive
4,According to the company 's updated strategy f...,positive


In [7]:
y = pd.get_dummies(raw.label)

In [8]:
df = pd.merge(raw, y, left_index=True, right_index=True)
df.drop(columns =["label"], inplace = True)

In [9]:
# Clean_text found on kaggle notebook
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [10]:
df['message'] = df['message'].map(lambda x : clean_text(x))

In [11]:
train, test = train_test_split(df, test_size=0.2)

In [12]:
train_sentences = train["message"].fillna("CVxTz").values
list_classes = ["negative", "neutral", "positive"]
train_y = train[list_classes].values

In [13]:
df.head()

,message,negative,neutral,positive
0,according to gran the company has no plans to ...,0,1,0
1,technopolis plans to develop in stages an area...,0,1,0
2,the international electronic industry company ...,1,0,0
3,with the new production plant the company woul...,0,0,1
4,according to the company updated strategy for ...,0,0,1


In [14]:
# Name of the BERT model to use
model_name = 'bert-base-uncased'

# Max length of tokens
max_length = 64

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
#config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
bert = TFAutoModel.from_pretrained(model_name)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [15]:
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32') 
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
x = bert.bert(inputs)

In [16]:
x

TFBaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                                 <KerasTensor: shape=(None, 64, 768) dtype=float32 (created by layer 'bert')>),
                                                ('pooler_output',
                                                 <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'bert')>)])

In [17]:
#x2 =Dense(512, activation='relu')(x[1])
x2 = GlobalAveragePooling1D()(x[0])
#x3 = Dropout(0.4)(x2)
#x4 = Dense(256, activation='relu')(x3)
#x5 = Dropout(0.5)(x4)
y = Dense(len(list_classes), activation='softmax', name='outputs')(x2)

model = Model(inputs=inputs, outputs=y)
#model.layers[2].trainable = False

# Take a look at the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 64)]         0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 64)]         0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 64,                                            

In [18]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [19]:
optimizer = Adam(lr=1e-5, decay=1e-6)
model.compile(loss = keras.losses.CategoricalCrossentropy(),
                  optimizer=optimizer,
                  metrics=['accuracy',f1_m])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
x = tokenizer(
    text=list(train_sentences),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [21]:
history = model.fit(
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    #x={'input_ids': x['input_ids']},
    y={'outputs': train_y},
    validation_split=0.2,
    batch_size=32,
    epochs=1)

97/97 [==============================] - 2233s 23s/step - loss: 0.7129 - accuracy: 0.6606 - f1_m: 0.6319 - val_loss: 0.4210 - val_accuracy: 0.8093 - val_f1_m: 0.8198


In [22]:
test_sentences = test["message"].fillna("CVxTz").values

In [23]:
test_x = tokenizer(
    text=list(test_sentences),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [24]:
del test_sentences
del x
del df

In [25]:
import gc
gc.collect()

1604

In [26]:
test_x

{'input_ids': <tf.Tensor: shape=(970, 64), dtype=int32, numpy=
array([[ 101, 2429, 2000, ...,    0,    0,    0],
       [ 101, 4341, 3445, ...,    0,    0,    0],
       [ 101, 1996, 3643, ...,    0,    0,    0],
       ...,
       [ 101, 2045, 2106, ...,    0,    0,    0],
       [ 101, 2004, 2172, ...,    0,    0,    0],
       [ 101, 6599, 3473, ...,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(970, 64), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [27]:
pred_y=model.predict(x={'input_ids': test_x['input_ids'], 'attention_mask': test_x['attention_mask']},batch_size=32)

In [28]:
df_test_y = test[["negative", "neutral", "positive"]]
test_y = df_test_y.to_numpy()

In [29]:
#submission=pd.DataFrame(pred_y,columns=list_classes)
m = np.zeros_like(pred_y)
m[np.arange(len(pred_y)), pred_y.argmax(1)] = 1

In [30]:
from sklearn.metrics import f1_score
f1_score(test_y, m, average='macro')

0.7903723459603632